# 0 Imports

In [8]:
import pickle

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from prophet    import Prophet

from IPython.display        import Image
from IPython.core.display   import HTML

In [9]:
# Supressão da notação científica.
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.6f}'.format)

## 0.1 Funções Suporte

In [10]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [12,6]
    plt.rcParams['font.size'] = 20
    
    display( HTML('<style>.container { width: 100% !important;} </style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False)
    
    sns.set()
    
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 0.2 Load Data

In [11]:
with open('../data/interim/3_0_feature_engineering.pkl', 'rb') as arquivo:
    df = pickle.load(arquivo)
df.sample(2)

,subscription_name,subscription_guid,date,resource_guid,service_name,service_type,service_region,service_resource,quantity,cost,maquina,year,month,day,week_of_year,year_week,year_month,dia_da_semana,fim_de_semana,custo_diario
46489,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2023-04-30,42e2a22f-fabf-41ea-a21e-e86bb3ff5799,Storage,Standard Page Blob,All,Disk Read Operations,0.288400,0.000096,unicin,2023,4,30,17,2023-17,2023-04,6,1,5.090174
27246,Microsoft Azure Sponsorship,82375e17-0dda-4790-b6e8-c565d4f08e87,2023-09-30,42e2a22f-fabf-41ea-a21e-e86bb3ff5799,Storage,Standard Page Blob,All,Disk Read Operations,0.288400,0.000096,portal,2023,9,30,39,2023-39,2023-09,5,1,6.163489


# 4.0 EDA pós F.E.

## 4.1 Entendimento de Negócio

### 4.1.1 Premissas

O valor

## 4.2 Análise Univariada

## 4.3 Análise Bivariada

## 4.4 Análise Multivariada